In [1]:
import pickle
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [2]:
s_data = pickle.load(open("sensitivity.p", "rb"))

In [3]:
key_names  = ["pol(t0)", "build(t0)", "coal(t0)", "solar(t0)",
'dels+',
 'delb+',
 'dels-',
 'delc-',
 'delb-',
 "pol_intensity", "pol_threshold", "pol_decay_param", 
 "urban_growth(t0)", "build_decom(t0)", "coal_extract(t0)", "solar_extract(t0)", "solar_inflow(t0)"  ]
keys = list(s_data.keys())

In [4]:
def get_high_low(time_step, concern, stock_or_flux, keys=keys, key_names=key_names):
    """
    time_step: 10, 25, 50, 100
    concern: coal, pollution, buildings, solar
    stock_or_flux: stock, flux
     """
    
    # extract the relevant data from sensitivity data 
    _dict = {}
    _dict["low"] = []
    _dict["original"] = []
    _dict["high"] = []
    for k in keys:
        _dict["low"].append(s_data[k][0.5][time_step][stock_or_flux][concern] )
        _dict["original"].append(s_data[k][1][time_step][stock_or_flux][concern])
        _dict["high"].append(s_data[k][1.5][time_step][stock_or_flux][concern])

    
    # only look at differences 
    result = {}
    result["low"] = [np.abs(i - j) for i, j in zip(_dict["low"], _dict["original"])]

    result["high"] = [np.abs(i - j) for i, j in zip(_dict["low"], _dict["high"])]

    result["low"]  = [-1*i for i in result["low"]]

    return result


In [5]:
def plot_value_over_time(concern, stock_or_flux, keys=keys, key_names=key_names):
    time_steps = [10, 25, 50, 100]
    keys = key_names

    fig = make_subplots(rows=1, cols=4, shared_yaxes=True, subplot_titles=[f"Year {t}" for t in time_steps])

    for ix, t in enumerate(time_steps):
        result = get_high_low(t, concern, stock_or_flux)
        fig.add_trace(go.Bar(
            y = key_names,
            x = result["low"],
            name = "Low",
            orientation="h",
            marker_color = "rgb( 158, 202, 225)",
            showlegend = False, 
        ), row = 1, col=ix+1)

        fig.add_trace(go.Bar(
            y = key_names,
            x = result["high"],
            name = "High",
            orientation="h",
            marker_color = "Crimson",
            showlegend = False, 
        ), row = 1, col=ix+1)

    fig.update_layout(title=f"{concern.title()} {stock_or_flux.title()}", barmode="overlay")

    return fig

In [6]:
concerns = ["buildings", "coal", "solar", "pollution"]
s_or_f = ["stock", "flux"]

In [12]:
for s in s_or_f:
    for c in concerns:
        fig = plot_value_over_time(c, s)
        fig.write_image(f"figures/{c}_{s}.png", width=1200, height=700)
        print(c,s)
    


buildings stock
coal stock
solar stock
pollution stock
buildings flux
coal flux
solar flux
pollution flux


In [8]:
fig = plot_value_over_time("buildings", "flux")
fig

In [11]:
fig.write_image(f"figures/fig1.png", width=1200, height=700)